In [ ]:
IMAGE_DIM=64

from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor, Resize, Compose
dataset = FashionMNIST('data/fashion-mnist', train=True, download=True, transform=Compose([Resize(IMAGE_DIM), ToTensor()]))

In [6]:
from models import Generator, Discriminator
from trainer import WGanTrainer, WGanGpTrainer
from torch.utils.data import DataLoader

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

g = Generator()
g._initialize_weights()
d = Discriminator()

# g_optim = torch.optim.Adam(g.parameters(), lr=0.0001, betas=(0.0, 0.9))
# d_optim = torch.optim.Adam(d.parameters(), lr=0.0001, betas=(0.0, 0.9))

g_optim = torch.optim.RMSprop(g.parameters(), lr=5e-5)
d_optim = torch.optim.RMSprop(d.parameters(), lr=5e-5)

trainer = WGanGpTrainer(
    generator=g,
    critic=d,
    gen_optimizer=g_optim,
    critic_optimizer=d_optim,
    latent_dimension=100,
    device=device,
)

data_loader = DataLoader(dataset, batch_size=64, shuffle=True)

trainer.train(data_loader, epochs=5)

Training...


Epoch [1/5]: 100%|█████████▉| 937/938 [03:08<00:00,  4.97it/s]


RuntimeError: The expanded size of the tensor (64) must match the existing size (32) at non-singleton dimension 2.  Target sizes: [32, 1, 64, 64].  Tensor sizes: [32, 1]

In [1]:
import matplotlib.pyplot as plt
trainer.plot_losses()
sample = trainer.sample_generator(1)[0].detach().cpu().numpy()
plt.matshow(sample[0])

NameError: name 'trainer' is not defined